# AST

In [4]:
import json
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Load the AST from a JSON file
json_file_path = "C:\\Users\\Cody\\Desktop\\FYP Project\\Code\\AI Model\\AST\\UR\\8_ast.json"  # Replace with your file path
with open(json_file_path, "r") as file:
    ast_data = json.load(file)

# Step 2: Recursive function to flatten the AST
def flatten_ast(node, features):
    if isinstance(node, dict):
        # Extract features like node type, name, visibility, etc.
        node_type = node.get("type", "Unknown")
        features["node_types"].append(node_type)
        
        # Extract specific properties based on node type
        if "name" in node:
            features["names"].append(node["name"])
        if "visibility" in node:
            features["visibilities"].append(node["visibility"])
        if "stateMutability" in node:
            features["mutabilities"].append(node["stateMutability"])
        if "typeName" in node:
            features["data_types"].append(node["typeName"].get("name", "Unknown"))
        
        # Traverse sub-nodes
        for key, value in node.items():
            if isinstance(value, dict) or isinstance(value, list):
                flatten_ast(value, features)
    elif isinstance(node, list):
        for item in node:
            flatten_ast(item, features)

# Step 3: Initialize feature storage
features = defaultdict(list)

# Step 4: Flatten the AST and extract features
flatten_ast(ast_data, features)

# Step 5: Vectorizing the features (simplified example using one-hot encoding)
# Convert list of node types to a single string of space-separated tokens
node_types_str = " ".join(features["node_types"])

# Vectorize the node types (you can do the same for other features like names, visibilities, etc.)
vectorizer = CountVectorizer()
node_type_vectors = vectorizer.fit_transform([node_types_str])

# Convert the sparse matrix to an array
node_type_vectors = node_type_vectors.toarray()

# Print the vectorized feature for node types
print(node_type_vectors)

# Now you can use the `node_type_vectors` as input features for your AI model.


[[ 16   7   4   3  56   2  19  11  12  75  18   2   3  12  38  25   1   1
    7   3 990   2  13]]


# Libraries Import

In [6]:
import os
import re
import tokenize
from io import BytesIO

from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from sklearn.base import BaseEstimator
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import joblib
from sklearn.ensemble import RandomForestClassifier
import numpy as np


In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing

In [9]:
# Preprocessing functions
def tokenize_code(code, file_path):
    tokens = []
    reader = BytesIO(code.encode('utf-8')).readline
    try:
        for toknum, tokval, _, _, _ in tokenize.tokenize(reader):
            if toknum != tokenize.ENCODING:
                tokens.append(tokval)
    except tokenize.TokenError:
        print("Error tokenizing code in file:", file_path)
        error_files.append(file_path)
    return tokens

def normalize_code(code):
    code = re.sub(r'//.*', '', code)
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    code = re.sub(r'\s+', ' ', code).strip()
    return code

def preprocess_code(code, file_path):
    normalized_code = normalize_code(code)
    tokens = tokenize_code(normalized_code, file_path)
    return ' '.join(tokens)

# Load data from directories
def load_data_from_directory(directory, label):
    data = []
    labels = []
    error_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".sol"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                try:
                    code = file.read()
                    preprocessed_code = preprocess_code(code, filepath)
                    data.append(preprocessed_code)
                    labels.append(label)
                except Exception:
                    print("Error processing file:", filepath)
                    error_files.append(filepath)
    with open('error_files.txt', 'w') as file:
        file.write('\n'.join(error_files))
    return data, labels


In [10]:
#Only Run this code on first time, then save the data and labels so next time can be loaded directly

# Paths to the directories
vulnerable_dir = './Contracts for training/reentrant_contracts'
non_vulnerable_dir = './Contracts for training/non duplicated data'

# Load and label the data
vulnerable_data, vulnerable_labels = load_data_from_directory(vulnerable_dir, 1)
non_vulnerable_data, non_vulnerable_labels = load_data_from_directory(non_vulnerable_dir, 0)
    
# Combine the data and labels
data = vulnerable_data + non_vulnerable_data
labels = vulnerable_labels + non_vulnerable_labels

Error tokenizing code in file: ./Contracts for training/reentrant_contracts\1055.sol_914.sol
Error processing file: ./Contracts for training/reentrant_contracts\1055.sol_914.sol
Error tokenizing code in file: ./Contracts for training/reentrant_contracts\31445.sol_59.sol
Error processing file: ./Contracts for training/reentrant_contracts\31445.sol_59.sol
Error tokenizing code in file: ./Contracts for training/reentrant_contracts\32054.sol_99.sol
Error processing file: ./Contracts for training/reentrant_contracts\32054.sol_99.sol
Error tokenizing code in file: ./Contracts for training/reentrant_contracts\35838.sol_718.sol
Error processing file: ./Contracts for training/reentrant_contracts\35838.sol_718.sol
Error tokenizing code in file: ./Contracts for training/reentrant_contracts\35875.sol_792.sol
Error processing file: ./Contracts for training/reentrant_contracts\35875.sol_792.sol
Error tokenizing code in file: ./Contracts for training/reentrant_contracts\36022.sol_803.sol
Error proces

## Save Preprocessed Data

In [11]:
# Paths to save the data and labels
data_path = './Preprocessed Data/preprocessed_data_correct.pkl' #File too huge, cannot be uploaded to github
labels_path = './Preprocessed Data/labels_correct.pkl'

# Save the preprocessed data
with open(data_path, 'wb') as data_file:
    pickle.dump(data, data_file)

# Save the labels
with open(labels_path, 'wb') as labels_file:
    pickle.dump(labels, labels_file)

## Load Saved Preprocessed Data

In [ ]:
# Paths to the saved data and labels
data_path = './Preprocessed Data/preprocessed_data_correct.pkl'
labels_path = './Preprocessed Data/labels_correct.pkl'

# Load the preprocessed data
with open(data_path, 'rb') as data_file:
    data = pickle.load(data_file)

# Load the labels
with open(labels_path, 'rb') as labels_file:
    labels = pickle.load(labels_file)

## Vectorizer

In [12]:
vectorizer = TfidfVectorizer(max_features=1000)

In [13]:
# Save the fitted vectorizer
joblib.dump(vectorizer, 'tfidf_vectors.pkl')

['tfidf_vectors.pkl']

## Neural Network Vectors

In [14]:
X_nn = vectorizer.fit_transform(data).toarray().astype('float32')
y_nn = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

## Tfid Vectors

In [15]:
X = vectorizer.fit_transform(data).toarray()
y = labels

# Feed Forward Neural Network

In [16]:
# Define the neural network model
class SmartContractVulnerabilityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1=256, hidden_dim2=128):
        super(SmartContractVulnerabilityModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
# Skorch wrapper for the PyTorch model
net = NeuralNetClassifier(
    SmartContractVulnerabilityModel,
    module__input_dim=1000,
    max_epochs=20,  # Higher number of epochs with early stopping
    lr=0.001,
    optimizer=optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    iterator_train__shuffle=True,
    callbacks=[EarlyStopping(patience=5)],  # Early stopping after 5 epochs without improvement
)

# Hyperparameter grid
params = {
    'lr': [0.001,0.01],
    'max_epochs': [20,10],
    'module__hidden_dim1': [256,128],
    'module__hidden_dim2': [128,64]
}

In [18]:
# Initialize GridSearchCV
#cv is for cross validation
#n_jobs=-1 means use all available cores
gs = GridSearchCV(net, params, refit=True, cv=5, scoring='accuracy')

# Perform grid search
gs.fit(X_nn, y_nn)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2963       0.8786        0.2789  4.4337
      2        0.2744       0.8794        0.2775  5.1951
      3        0.2693       0.8790        0.2785  4.8274
      4        0.2645       0.8784        0.2756  5.0134
      5        0.2595       0.8768        0.2759  6.0214
      6        0.2560       0.8752        0.2776  6.2751
      7        0.2519       0.8747        0.2812  7.2224
      8        0.2477       0.8703        0.2835  5.7274
Stopping since valid_loss has not improved in the last 5 epochs.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2972       0.8786        0.2890  5.4758
      2        0.2749       0.8714        0.2938  5.9287
      3        0.2695       0.8771        0.3011  7.0449
      4        0.2647       0.8765        0.3141  5.7234
      5        0.2601  

GridSearchCV(cv=5,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.SmartContractVulnerabilityModel'>,
  module__input_dim=1000,
),
             param_grid={'lr': [0.001, 0.01], 'max_epochs': [20, 10],
                         'module__hidden_dim1': [256, 128],
                         'module__hidden_dim2': [128, 64]},
             scoring='accuracy')

In [19]:
# Print the best parameters and the best score
print("Best parameters found:", gs.best_params_)
print("Best score:", gs.best_score_)

Best parameters found: {'lr': 0.01, 'max_epochs': 10, 'module__hidden_dim1': 128, 'module__hidden_dim2': 128}
Best score: 0.878087182522638


In [20]:
# Save the best model
torch.save(gs.best_estimator_.module_.state_dict(), 'neural_network_model_modified.pth')

In [21]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_nn, y_nn, test_size=0.2, random_state=42)

# Use the best model found by GridSearchCV
best_model = gs.best_estimator_

# Fit the best model on the entire training set
best_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

# Print detailed classification report
print(classification_report(y_test, y_pred))


Re-initializing module because the following parameters were re-set: hidden_dim1, hidden_dim2, input_dim.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2893       0.8791        0.2801  7.3981
      2        0.2746       0.8797        0.2767  9.0191
      3        0.2699       0.8806        0.2765  7.7918
      4        0.2643       0.8800        0.2787  7.5754
      5        0.2602       0.8804        0.2787  6.8337
      6        0.2566       0.8789        0.2790  7.0740
      7        0.2537       0.8791        0.2876  7.0027
Stopping since valid_loss has not improved in the last 5 epochs.
Test Accuracy: 0.8796
Test Precision: 0.4906
Test Recall: 0.0525
Test F1 Score: 0.0949
              precision    recall  f1-score   support

         0.0       0.88      0.99      0.94     25380
         1.0       0.49      0.05      0.09      3466

    accurac

# Random Forest

In [22]:
# Define the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [23]:
params_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Initialize GridSearchCV
gs_rf  = GridSearchCV(
    rf_model, 
    params_rf, 
    refit=True, 
    cv=3, 
    scoring='accuracy',
    verbose=2 
)

In [24]:
gs_rf.fit(X, y)

# Print the best parameters and the best score
print("Best parameters found:", gs_rf.best_params_)
print("Best score:", gs_rf.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time= 1.3min
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time= 1.0min
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time= 1.0min
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time= 2.0min
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time= 2.0min
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time= 2.0min
[CV] END max_depth=None, min_samples_split=5, n_estimators=50; total time=  52.7s
[CV] END max_depth=None, min_samples_split=5, n_estimators=50; total time=  51.4s
[CV] END max_depth=None, min_samples_split=5, n_estimators=50; total time=  50.0s
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time= 1.7min
[CV] END max_depth=None, min_samples_split=5, n_estimators=100; total time= 1.8min
[CV] END max_depth=None, min_sam

In [25]:
# Save the best model
joblib.dump(gs_rf.best_estimator_, 'random_forest_model_modified.pkl')

['random_forest_model_modified.pkl']

In [26]:
# Use the best model found by GridSearchCV for Random Forest
best_rf_model = gs_rf.best_estimator_
best_rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

c:\Users\Cody\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [27]:
print(f"Random Forest - Test Accuracy: {accuracy_rf:.4f}")
print(f"Random Forest - Test Precision: {precision_rf:.4f}")
print(f"Random Forest - Test Recall: {recall_rf:.4f}")
print(f"Random Forest - Test F1 Score: {f1_rf:.4f}")

print(classification_report(y_test, y_pred_rf))

Random Forest - Test Accuracy: 0.8806
Random Forest - Test Precision: 0.5714
Random Forest - Test Recall: 0.0242
Random Forest - Test F1 Score: 0.0465
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94     25380
         1.0       0.57      0.02      0.05      3466

    accuracy                           0.88     28846
   macro avg       0.73      0.51      0.49     28846
weighted avg       0.84      0.88      0.83     28846



# Graph Neural Network

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

In [29]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return torch.log_softmax(x, dim=1)

# Wrap the GNN model for use with Skorch
class SkorchGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SkorchGCN, self).__init__()
        self.model = GCN(input_dim, hidden_dim1, hidden_dim2, output_dim)
        
    def forward(self, x):
        edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
        data = Data(x=x, edge_index=edge_index)
        return self.model(data)

In [30]:
net = NeuralNetClassifier(
    SkorchGCN,
    module__input_dim=X.shape[1],
    module__hidden_dim1=64,
    module__hidden_dim2=32,
    module__output_dim=2,
    max_epochs=20,
    lr=0.01,
    optimizer=optim.Adam,
    criterion=nn.CrossEntropyLoss,
    iterator_train__shuffle=True,
    callbacks=[EarlyStopping(patience=5)],
)

# Hyperparameter grid
params_gnn = {
    'lr': [0.01, 0.001],
    'max_epochs': [10, 20],
    'module__hidden_dim1': [64,128],
    'module__hidden_dim2': [16, 32, 64],
}

In [31]:
# Initialize GridSearchCV
gs_gnn = GridSearchCV(net, params_gnn, refit=True, cv=3, scoring='accuracy', verbose=2)

In [32]:
# Perform grid search
gs_gnn.fit(X_train, y_train)

print("Best parameters found:", gs_gnn.best_params_)
print("Best score:", gs_gnn.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2942       0.8788        0.2827  4.3107
      2        0.2762       0.8786        0.2804  5.7833
      3        0.2706       0.8773        0.2804  5.0873
      4        0.2655       0.8796        0.2797  5.1404
      5        0.2609       0.8796        0.2857  4.8928
      6        0.2563       0.8790        0.2866  4.9176
      7        0.2535       0.8762        0.2900  4.8489
      8        0.2500       0.8797        0.3027  4.8370
Stopping since valid_loss has not improved in the last 5 epochs.
[CV] END lr=0.01, max_epochs=10, module__hidden_dim1=64, module__hidden_dim2=16; total time=  46.8s
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2935       0.8793        0.2822  4.5593
      2        0.2774       0.880

In [33]:
# Save the best model
torch.save(gs_gnn.best_estimator_.module_.state_dict(), 'gnn_best_model_modified.pth')

In [34]:
# Use the best model found by GridSearchCV
gs_gnn_best_model = gs_gnn.best_estimator_

# Fit the best model on the entire training set
gs_gnn_best_model.fit(X_train, y_train)

# Predict on the test set
y_pred = gs_gnn_best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

Re-initializing module because the following parameters were re-set: hidden_dim1, hidden_dim2, input_dim, output_dim.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2896       0.8793        0.2813  6.8272
      2        0.2765       0.8795        0.2766  7.3106
      3        0.2709       0.8800        0.2755  7.7024
      4        0.2662       0.8799        0.2771  7.1115
      5        0.2629       0.8804        0.2788  7.1660
      6        0.2588       0.8773        0.2781  7.2298
      7        0.2563       0.8791        0.2811  7.5271
Stopping since valid_loss has not improved in the last 5 epochs.


In [35]:
print(f"GNN - Test Accuracy: {accuracy:.4f}")
print(f"GNN - Test Precision: {precision:.4f}")
print(f"GNN - Test Recall: {recall:.4f}")
print(f"GNN - Test F1 Score: {f1:.4f}")
print(classification_report(y_test, y_pred))

GNN - Test Accuracy: 0.8805
GNN - Test Precision: 0.5581
GNN - Test Recall: 0.0277
GNN - Test F1 Score: 0.0528
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     25380
           1       0.56      0.03      0.05      3466

    accuracy                           0.88     28846
   macro avg       0.72      0.51      0.49     28846
weighted avg       0.84      0.88      0.83     28846



# Back Propagation Neural Network

In [36]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [37]:
class BPNNContractVulnerabilityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1=256):
        super(BPNNContractVulnerabilityModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x).squeeze(1)

In [38]:
input_dim = X.shape[1]
net = NeuralNetClassifier(
    module=BPNNContractVulnerabilityModel,
    module__input_dim=input_dim,
    max_epochs=20,
    lr=0.01,
    optimizer=optim.Adam,
    criterion=nn.BCELoss,
    iterator_train__shuffle=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    callbacks=[EarlyStopping(patience=5)]
)

# Define parameter grid for grid search
params_bpnn = {
    'lr': [0.01, 0.001],
    'module__hidden_dim1': [128, 256, 512],
    'max_epochs': [10, 20]
}

In [39]:
gs_bpnn = GridSearchCV(net, params_bpnn, cv=3, scoring='accuracy', verbose=2)


In [40]:
gs_bpnn.fit(X_train, y_train)

# Use the best model found by GridSearchCV
best_model = gs_bpnn.best_estimator_

# Print best parameters and best score
print("Best parameters found:", gs_bpnn.best_params_)
print("Best score:", gs_bpnn.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2928       0.8784        0.2828  3.3613
      2        0.2757       0.8784        0.2786  3.4947
      3        0.2705       0.8781        0.2805  3.8034
      4        0.2660       0.8796        0.2810  4.8663
      5        0.2617       0.8763        0.2800  4.0131
      6        0.2580       0.8770        0.2839  4.2512
Stopping since valid_loss has not improved in the last 5 epochs.
[CV] END ....lr=0.01, max_epochs=10, module__hidden_dim1=128; total time=  28.6s
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.2943       0.8790        0.2856  2.9836
      2        0.2771       0.8787        0.2770  3.5114
      3        0.2723       0.8790        0.2812  3.7774
      4        0.2680       0.8792        0.2789  3.

In [41]:
torch.save(gs_bpnn.best_estimator_.module_.state_dict(), 'bpnn_best_model_modified.pth')

In [42]:
y_pred = best_model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)
acc = accuracy_score(y_test.numpy(), y_pred_class)
precision = precision_score(y_test.numpy(), y_pred_class)
recall = recall_score(y_test.numpy(), y_pred_class)
f1 = f1_score(y_test.numpy(), y_pred_class)

In [43]:
print(f"Back Propagation Neural Network - Test Accuracy: {acc:.4f}")
print(f"Back Propagation Neural Network - Test Precision: {precision:.4f}")
print(f"Back Propagation Neural Network - Test Recall: {recall:.4f}")
print(f"Back Propagation Neural Network - Test F1 Score: {f1:.4f}")
print(classification_report(y_test.numpy(), y_pred_class))

Back Propagation Neural Network - Test Accuracy: 0.8794
Back Propagation Neural Network - Test Precision: 0.4959
Back Propagation Neural Network - Test Recall: 0.2077
Back Propagation Neural Network - Test F1 Score: 0.2928
              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93     25380
         1.0       0.50      0.21      0.29      3466

    accuracy                           0.88     28846
   macro avg       0.70      0.59      0.61     28846
weighted avg       0.85      0.88      0.86     28846

